# Import

In [760]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.metrics import mean_absolute_error 
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor, BaggingRegressor, GradientBoostingRegressor
from xgboost import XGBRFRegressor
import warnings 
warnings.filterwarnings(action='ignore')

# 데이터 불러오기

In [655]:
# 코랩으로 불러올 파일의 마우스 우클릭으로 경로 복사를 통해 불러옴. 
train_set = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/iris_train.csv')
test_set = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/iris_test.csv')

# 데이터 탐색

In [656]:
train_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75 entries, 0 to 74
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 75 non-null     int64  
 1   species            75 non-null     object 
 2   sepal length (cm)  75 non-null     float64
 3   petal length (cm)  75 non-null     float64
 4   sepal width (cm)   75 non-null     float64
 5   petal width (cm)   75 non-null     float64
dtypes: float64(4), int64(1), object(1)
memory usage: 3.6+ KB


In [657]:
# count : 개수 / mean : 평균 / std : 표준편차 
train_set.describe()

,id,sepal length (cm),petal length (cm),sepal width (cm),petal width (cm)
count,75.000000,75.000000,75.000000,75.000000,75.000000
mean,37.000000,6.044000,4.258667,2.958667,1.384000
std,21.794495,0.842653,1.615390,0.388376,0.697106
min,0.000000,4.300000,1.100000,2.000000,0.100000
25%,18.500000,5.550000,3.750000,2.800000,1.000000
50%,37.000000,6.100000,4.700000,3.000000,1.500000
75%,55.500000,6.600000,5.300000,3.200000,1.850000
max,74.000000,7.700000,6.900000,4.000000,2.500000


# 데이터 전처리

In [658]:
# 데이터의 구분자로 활용되는 id 항목을 제외시킴
train_set.drop(['id'], axis = 1, inplace = True)
test_set.drop(['id'], axis = 1, inplace = True)

In [659]:
# 명목형 자료 항목 변환에는 1) 수치 매핑 기법, 2) 더미 기법이 있음
# 1) 수치 매핑 기법 채택
train_set.replace({'setosa' : 1, 'versicolor' : 2, 'virginica' : 3}, inplace=True)
test_set.replace({'setosa' : 1, 'versicolor' : 2, 'virginica' : 3}, inplace=True)

In [660]:
train_set

,species,sepal length (cm),petal length (cm),sepal width (cm),petal width (cm)
0,1,4.4,1.4,2.9,0.2
1,2,6.4,4.5,3.2,1.5
2,3,6.2,4.8,2.8,1.8
3,3,7.2,6.1,3.6,2.5
4,1,4.9,1.4,3.0,0.2
...,...,...,...,...,...
70,2,6.5,4.6,2.8,1.5
71,2,5.6,3.6,2.9,1.3
72,2,6.2,4.5,2.2,1.5
73,2,4.9,3.3,2.4,1.0


In [661]:
test_set

,species,sepal length (cm),petal length (cm)
0,1,5.4,1.7
1,1,5.7,1.5
2,1,5.3,1.5
3,1,5.1,1.9
4,3,6.0,4.8
...,...,...,...
70,2,5.8,4.0
71,1,4.4,1.3
72,3,6.3,5.6
73,2,5.1,3.0


## 접근 방식 선정
### 1. 꽃잎, 꽃받침 너비 두 개를 같이 예측하는 "다변량 회귀"
### 2. 꽃잎, 꽃받침 너비 각각을 따로 예측하는 "단변량 회귀"

# 1.다변량회귀
- 기본적으로 지원하는 트리계열 모델 중 의사결정나무, 랜덤포레스트가 있음
- 나머지 모델에 있어서는 다변량회귀를 지원해 주는 함수를 사용해야함.
- 하루 제출 3번이라는 제약이 있어 "단변량 회귀"의 성능이 더 좋을 것이라 판단해 기본으로 제공하는 모델만 이용함.

In [662]:
# 데이터 분리
X = train_set[['species', 'sepal length (cm)', 'petal length (cm)']]
Y = train_set[['sepal width (cm)', 'petal width (cm)']] 

In [700]:
# 의사결정나무(decision tree)

In [681]:
# validation_set : training을 한 후에 만들어진 모형이 잘 예측을 하는지 그 성능을 평가하기 위해서 사용
X_train, X_val, y_train, y_val = train_test_split(X, Y, test_size = 0.2)

In [682]:
dt = DecisionTreeRegressor()

In [683]:
dt.fit(X_train, y_train)

DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=None,
                      max_features=None, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=None, splitter='best')

In [684]:
predict_dt = dt.predict(X_val)

In [685]:
print(f'MAE : {mean_absolute_error(y_val, predict_dt): .4f}')

MAE :  0.2867


dt.predict(test_set) → MAE : 0.215

In [701]:
# 랜덤 포레스트(RandomForestRegressor)

In [692]:
X_train, X_val, y_train, y_val = train_test_split(X, Y)

In [693]:
# n_estimators : 의사결정나무의 개수 지정, 기본값 = 100
rf = RandomForestRegressor(n_estimators=1000)

In [695]:
rf.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=1000, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [698]:
predict_rf = rf.predict(X_val)

In [699]:
print(f'MAE : {mean_absolute_error(y_val, predict_rf): .4f}')

MAE :  0.2146


rf.predict(test_set) → MAE : 0.183

# 2.단변량회귀
- 종속변수를 각각 하나씩 나눠 진행, 즉 모델 두 개로 각각 예측해 취합함

In [702]:
train_sepal = train_set[['species', 'sepal length (cm)', 'sepal width (cm)']]
train_petal = train_set[['species', 'petal length (cm)', 'petal width (cm)']]

test_sepal = test_set[['species',	'sepal length (cm)']]
test_petal = test_set[['species',	'petal length (cm)']]

In [703]:
X_s = train_sepal[['species', 'sepal length (cm)']]
y_s = train_sepal[['sepal width (cm)']]

X_p = train_petal[['species', 'petal length (cm)']]
y_p = train_petal[['petal width (cm)']]

In [749]:
# random_state 값을 바꿔가며 validation_set을 이용해 평가하는 과정에서 가장 적은 MAE 출력하는 값을 넣음
X_train_s, X_val_s, y_train_s, y_val_s = train_test_split(X_s, y_s, test_size = 0.2, random_state=50)
X_train_p, X_val_p, y_train_p, y_val_p = train_test_split(X_p, y_p, test_size = 0.2, random_state=42)

- 단변량회귀를 진행하는데 있어 가장 좋은 모델은 서포트벡터머신 이었음.
- 클래스 사이의 경계에 위치한 데이터 포인트를 서포트 벡터라고함.
- 각 서포트 벡터가 클래스 사이의 결정 경계를 구분하는데 얼마나 중요한지를 학습.

## 매개변수 튜닝
- SVM은 입력 데이터가 정규화 되어야 좋은 성능을 보임
- 주로 모든 특성 값을 [0, 1] 범위로 맞추는 방법 사용
- StandardScaler(), MinMaxScaler() 이용

### sepal

In [728]:
pipline = Pipeline([('scaler',StandardScaler()), ('svm', SVR()) ])
pipline.fit(X_train_s, y_train_s)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Pipeline(memory=None,
         steps=[('scaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('svm',
                 SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1,
                     gamma='scale', kernel='rbf', max_iter=-1, shrinking=True,
                     tol=0.001, verbose=False))],
         verbose=False)

In [761]:
# C = 정규화 매개 변수, 강도는 C에 반비례, L2 페널티(릿지)
# gamma = 가우시안 함수의 표준편차와 관련되어 있는데, 클수록 작은 표준편차를 가짐.
params = {
            'svm__C'     : [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000],
            'svm__gamma' : [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000] 
         }
grid = GridSearchCV(pipline, param_grid = params, cv = 5, scoring = 'neg_mean_absolute_error', verbose=2)
grid.fit(X_train_s, y_train_s)

Fitting 5 folds for each of 81 candidates, totalling 405 fits
[CV] svm__C=0.0001, svm__gamma=0.0001 ................................
[CV] ................. svm__C=0.0001, svm__gamma=0.0001, total=   0.0s
[CV] svm__C=0.0001, svm__gamma=0.0001 ................................
[CV] ................. svm__C=0.0001, svm__gamma=0.0001, total=   0.0s
[CV] svm__C=0.0001, svm__gamma=0.0001 ................................
[CV] ................. svm__C=0.0001, svm__gamma=0.0001, total=   0.0s
[CV] svm__C=0.0001, svm__gamma=0.0001 ................................
[CV] ................. svm__C=0.0001, svm__gamma=0.0001, total=   0.0s
[CV] svm__C=0.0001, svm__gamma=0.0001 ................................
[CV] ................. svm__C=0.0001, svm__gamma=0.0001, total=   0.0s
[CV] svm__C=0.0001, svm__gamma=0.001 .................................
[CV] .................. svm__C=0.0001, svm__gamma=0.001, total=   0.0s
[CV] svm__C=0.0001, svm__gamma=0.001 .................................
[CV] ..........

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s


[CV] ..................... svm__C=0.0001, svm__gamma=10, total=   0.0s
[CV] svm__C=0.0001, svm__gamma=100 ...................................
[CV] .................... svm__C=0.0001, svm__gamma=100, total=   0.0s
[CV] svm__C=0.0001, svm__gamma=100 ...................................
[CV] .................... svm__C=0.0001, svm__gamma=100, total=   0.0s
[CV] svm__C=0.0001, svm__gamma=100 ...................................
[CV] .................... svm__C=0.0001, svm__gamma=100, total=   0.0s
[CV] svm__C=0.0001, svm__gamma=100 ...................................
[CV] .................... svm__C=0.0001, svm__gamma=100, total=   0.0s
[CV] svm__C=0.0001, svm__gamma=100 ...................................
[CV] .................... svm__C=0.0001, svm__gamma=100, total=   0.0s
[CV] svm__C=0.0001, svm__gamma=1000 ..................................
[CV] ................... svm__C=0.0001, svm__gamma=1000, total=   0.0s
[CV] svm__C=0.0001, svm__gamma=1000 ..................................
[CV] .

[Parallel(n_jobs=1)]: Done 405 out of 405 | elapsed:    3.7s finished


GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('scaler',
                                        StandardScaler(copy=True,
                                                       with_mean=True,
                                                       with_std=True)),
                                       ('svm',
                                        SVR(C=1.0, cache_size=200, coef0=0.0,
                                            degree=3, epsilon=0.1,
                                            gamma='scale', kernel='rbf',
                                            max_iter=-1, shrinking=True,
                                            tol=0.001, verbose=False))],
                                verbose=False),
             iid='deprecated', n_jobs=None,
             param_grid={'svm__C': [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000,
                                    10000],
                         'svm__ga

In [730]:
grid.best_params_

{'svm__C': 10, 'svm__gamma': 0.1}

In [750]:
svm = make_pipeline(MinMaxScaler(), SVR(C = 10, gamma = 0.1))

In [751]:
svm.fit(X_train_s, y_train_s)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Pipeline(memory=None,
         steps=[('minmaxscaler', MinMaxScaler(copy=True, feature_range=(0, 1))),
                ('svr',
                 SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1,
                     gamma='scale', kernel='rbf', max_iter=-1, shrinking=True,
                     tol=0.001, verbose=False))],
         verbose=False)

In [752]:
predict_svm_s = svm.predict(X_val_s)

In [753]:
print(f'MAE : {mean_absolute_error(y_val_s, predict_svm_s): .4f}')

MAE :  0.1651


### petal

In [737]:
pipline = Pipeline([('scaler',StandardScaler()), ('svm', SVR()) ])
pipline.fit(X_train_p, y_train_p)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Pipeline(memory=None,
         steps=[('scaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('svm',
                 SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1,
                     gamma='scale', kernel='rbf', max_iter=-1, shrinking=True,
                     tol=0.001, verbose=False))],
         verbose=False)

In [762]:
params = {
            'svm__C'     : [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000],
            'svm__gamma' : [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000] 
         }
grid = GridSearchCV(pipline, param_grid = params, cv = 5, scoring = 'neg_mean_absolute_error', verbose=2)
grid.fit(X_train_p, y_train_p)

Fitting 5 folds for each of 81 candidates, totalling 405 fits
[CV] svm__C=0.0001, svm__gamma=0.0001 ................................
[CV] ................. svm__C=0.0001, svm__gamma=0.0001, total=   0.0s
[CV] svm__C=0.0001, svm__gamma=0.0001 ................................
[CV] ................. svm__C=0.0001, svm__gamma=0.0001, total=   0.0s
[CV] svm__C=0.0001, svm__gamma=0.0001 ................................
[CV] ................. svm__C=0.0001, svm__gamma=0.0001, total=   0.0s
[CV] svm__C=0.0001, svm__gamma=0.0001 ................................
[CV] ................. svm__C=0.0001, svm__gamma=0.0001, total=   0.0s
[CV] svm__C=0.0001, svm__gamma=0.0001 ................................
[CV] ................. svm__C=0.0001, svm__gamma=0.0001, total=   0.0s
[CV] svm__C=0.0001, svm__gamma=0.001 .................................
[CV] .................. svm__C=0.0001, svm__gamma=0.001, total=   0.0s
[CV] svm__C=0.0001, svm__gamma=0.001 .................................
[CV] ..........

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s


[CV] .................... svm__C=0.0001, svm__gamma=100, total=   0.0s
[CV] svm__C=0.0001, svm__gamma=100 ...................................
[CV] .................... svm__C=0.0001, svm__gamma=100, total=   0.0s
[CV] svm__C=0.0001, svm__gamma=100 ...................................
[CV] .................... svm__C=0.0001, svm__gamma=100, total=   0.0s
[CV] svm__C=0.0001, svm__gamma=100 ...................................
[CV] .................... svm__C=0.0001, svm__gamma=100, total=   0.0s
[CV] svm__C=0.0001, svm__gamma=100 ...................................
[CV] .................... svm__C=0.0001, svm__gamma=100, total=   0.0s
[CV] svm__C=0.0001, svm__gamma=1000 ..................................
[CV] ................... svm__C=0.0001, svm__gamma=1000, total=   0.0s
[CV] svm__C=0.0001, svm__gamma=1000 ..................................
[CV] ................... svm__C=0.0001, svm__gamma=1000, total=   0.0s
[CV] svm__C=0.0001, svm__gamma=1000 ..................................
[CV] .

[Parallel(n_jobs=1)]: Done 405 out of 405 | elapsed:    3.9s finished


GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('scaler',
                                        StandardScaler(copy=True,
                                                       with_mean=True,
                                                       with_std=True)),
                                       ('svm',
                                        SVR(C=1.0, cache_size=200, coef0=0.0,
                                            degree=3, epsilon=0.1,
                                            gamma='scale', kernel='rbf',
                                            max_iter=-1, shrinking=True,
                                            tol=0.001, verbose=False))],
                                verbose=False),
             iid='deprecated', n_jobs=None,
             param_grid={'svm__C': [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000,
                                    10000],
                         'svm__ga

In [739]:
grid.best_params_

{'svm__C': 100, 'svm__gamma': 0.01}

In [754]:
svm = make_pipeline(StandardScaler(), SVR(C=100, gamma=0.01))

In [755]:
svm.fit(X_train_p, y_train_p)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Pipeline(memory=None,
         steps=[('standardscaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('svr',
                 SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1,
                     gamma='scale', kernel='rbf', max_iter=-1, shrinking=True,
                     tol=0.001, verbose=False))],
         verbose=False)

In [756]:
predict_svm_p = svm.predict(X_val_p)

In [757]:
print(f'MAE : {mean_absolute_error(y_val_p, predict_svm_p): .4f}')

MAE :  0.0935


- svm.predict(test_sepal)
- svm.predict(test_petal)

# MAE : 0.17042